## Урок 3 

### Задание начального уровня

Потренируйтесь в нахождении матрицы схожести для валидационного сета

* загрузите `brand_tweets_valid.csv`
* примените объект `vectorizer`, обученный на датасете `brand_tweets.csv` (просто скопируйте этот код из урока)
* примените функцию `pairwise_distances` к полученной матрице

In [10]:
import pandas as pd
import nltk
import string
from itertools import chain
import numpy as np

df_valid = pd.read_csv('sample_data/brand_tweets_valid.csv', sep=',', encoding='utf8')
# удаляем строки, в которых отсутствует текст твита
df_valid.drop(df_valid[df_valid.tweet_text.isnull()].index, inplace=True)

nltk.download('punkt', download_dir='.')


df_valid.head()



[nltk_data] Downloading package punkt to ....
[nltk_data]   Package punkt is already up-to-date!


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,Wow! Google maps for mobile v5 demo at #sxsw. ...,Other Google product or service,Positive emotion
1,The #google name was built on gettinng stuff o...,Google,Positive emotion
2,&quot;Apple opening a temporary store in Austi...,NaN,No emotion toward brand or product
3,#tech Apple Opening Pop-Up Store In Austin For...,Apple,Positive emotion
4,GSDM Google party is off the hook! #SXSW {link},Google,Positive emotion


In [7]:
stop_words = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd",
    'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
    'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
    'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
    'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out',
    'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not',
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'shold',
    "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't",
    'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
    'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
    'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
]

In [8]:
def tokenize_text(raw_text: str):
    """Функция для токенизации текста
    :param raw_text: исходная текстовая строка
    """
    tokenized_str = nltk.word_tokenize(raw_text)
    tokens = [i.lower() for i in tokenized_str if ( i not in string.punctuation )]
    filtered_tokens = [i for i in tokens if ( i not in stop_words )]
    return filtered_tokens

# применяем функцию в датафрейму с помощью метода .apply()
tokenized_tweets= df_valid.tweet_text.apply(tokenize_text)

# добавляем новую колонку в исходный датафрейм
df_valid = df_valid.assign(
    tokenized=tokenized_tweets
)

df_valid.tokenized.head()

0    [wow, google, maps, mobile, v5, demo, sxsw, nice]
1    [google, name, built, gettinng, stuff, trying,...
2    [quot, apple, opening, temporary, store, austi...
3    [tech, apple, opening, pop-up, store, austin, ...
4              [gsdm, google, party, hook, sxsw, link]
Name: tokenized, dtype: object

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
# инициализируем объект, который токенизирует наш текст
# в качестве единственного аргимента передаём функцию, которую мы написали в Уроке 2
# на разбивает каждый документ на токены
vectorizer = CountVectorizer(tokenizer=tokenize_text)
# применяем наш объект-токенизатор к датафрейму с твитами
document_matrix = vectorizer.fit_transform(df.tweet_text.values)
# результат - матрица, в которой находятся числа, строк в мастрице столько, сколько документов
# а столбцов столько, сколько токенов
document_matrix

<402x1765 sparse matrix of type '<class 'numpy.int64'>'
	with 4922 stored elements in Compressed Sparse Row format>

### Задание среднего уровня

Пользуясь матрицей схожести, полученной на предыдущем этапе, найдите top-5 твитов, похожих на твит валидационного сета с `id=14`.

In [21]:
test_tweet_index = 14

print(df_valid.iloc[test_tweet_index].tweet_text+'\n------------------------------\n')



from sklearn.metrics import pairwise_distances

tweet_distance = 1-pairwise_distances(document_matrix, metric="cosine")

print(tweet_distance)

Popup Apple Store crew has been giving out water to the people in line but they are in street clothes. No Apple logos anywhere yet. #SXSW
------------------------------

[[1.         0.21320072 0.17149859 ... 0.18898224 0.16222142 0.21320072]
 [0.21320072 1.         0.14625448 ... 0.24174689 0.20751434 0.18181818]
 [0.17149859 0.14625448 1.         ... 0.12964074 0.27820744 0.14625448]
 ...
 [0.18898224 0.24174689 0.12964074 ... 1.         0.1839418  0.16116459]
 [0.16222142 0.20751434 0.27820744 ... 0.1839418  1.         0.13834289]
 [0.21320072 0.18181818 0.14625448 ... 0.16116459 0.13834289 1.        ]]


In [39]:
import numpy as np

test_tweet_index = 14
# отсортируем твиты по “похожести” - чем похожее на source_tweet_index,
# тем ближе к началу списка sorted_similarity
sorted_similarity = np.argsort(-tweet_distance[test_tweet_index,:])

print(sorted_similarity)

[ 14 132  16 163   6 341  65  23 226  72  63 357 395  62 319 192   3  15
 238 121  12 390  55  24 205 260 129  51 297  64 206 279  67   2 158 313
 203 231 118 134  22 212 377 400 321 351 131 135 186  99 293 292 310 204
  13  21 384   5  20  68 358 354 207  78 225  31 224 123 287  53 120 331
 275 109 136 381  29  69  58 288 328 296 133 330  88  17 259 380  35 215
 352 314  49  42  28 300 219  98  71  81  47 301 361 138 139 195 150 200
 343 190 342 329 348 211 383 148 147 335  79 336 245 291  18 235 122 295
 240 251 359  66 389  33  27 216 199  19 246 168 227 145   4 157  75 276
 127 391 272 385 156 345 114 210 100 282  36 265 155 170 277 278 311 386
 247 375 376 180 217   0  87  73  70 151 167 124 119 374 198  91 160 125
 298  85  84 281 337 197 344 347 364 254 369 261 105 258 255 142 228 284
 289 173 339 185 175 214  97  56 346 285 252 115 308 379  30   1 318 401
 201 140 179 213 176  74  61 290  95 101 104 256  46 356  41 316 271 257
 263   8 253  86 268  52  93 388  50 340 184 368  5

In [31]:
print(df_valid.iloc[14].tweet_text)
print('-------------')
print(df_valid.iloc[sorted_similarity[1]].tweet_text)
print('-------------')
print(df_valid.iloc[sorted_similarity[2]].tweet_text)
print('-------------')
print(df_valid.iloc[sorted_similarity[3]].tweet_text)

Popup Apple Store crew has been giving out water to the people in line but they are in street clothes. No Apple logos anywhere yet. #SXSW
-------------
Apple employees just showed up in force to the #SXSW PopUp Apple Store. #iPad2
-------------
#sxsw apple store run out for the day :( boo apple.
-------------
video from the popup Apple store: {link} #sxsw #sxswi


### Задание высокого уровня.

У вас есть матрица схожести между объектами. Попробуйте решить задачу поиска дубликатов в тексте

 Напишите функцию на Python, которая принимает индекс твита, пороговое значение (число от $0.0$ до $1.0$ и матрицу схожести, а затем выводит все твиты, схожесть которых больше, чем пороговое значение.

In [61]:
def search_dublicat(index_tweet, level_distance, matrix_distance, data_frame):
  raw_distance = matrix_distance[index_tweet]
  sorted_raw = raw_distance >level_distance
  print(df_valid.tweet_text[sorted_raw])

search_dublicat(6, 0.7, tweet_distance, df_valid)


6    #SXSW Apple Pop Up Store still has iPads in st...
Name: tweet_text, dtype: object
